# Generate wordvecs for the tweets

## Loading data

In [1]:
import pandas as pd
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')
train.head()

,id,keyword,location,text,target,cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...


## Add wordvecs

In [2]:
import spacy
# Load a larger model with vectors
nlp = spacy.load("en_core_web_lg")

Remove punctuation

In [3]:
import string
table = str.maketrans('', '', string.punctuation)
train['cleaned_text'] = train['cleaned_text'].str.translate(table).str.strip()
test['cleaned_text'] = test['cleaned_text'].str.translate(table).str.strip()

Remove whitespace

In [4]:
for _ in range(280):
    train['cleaned_text'] = train['cleaned_text'].str.replace('  ', ' ')
    test['cleaned_text'] = test['cleaned_text'].str.replace('  ', ' ')

Insert keywords into text

In [5]:
train['keyword'].fillna('',inplace=True)
test['keyword'].fillna('',inplace=True)

In [6]:
train['cleaned_text_keyword'] = (train['keyword'] + ' ' + train['cleaned_text']).str.strip()
test['cleaned_text_keyword'] = (test['keyword'] + ' ' + test['cleaned_text']).str.strip()

Get wordvecs

In [7]:
train['nlp'] = train['cleaned_text'].apply(lambda s: nlp(s))
train['wordvec'] = train['nlp'].apply(lambda s: s.vector)
test['nlp'] = test['cleaned_text'].apply(lambda s: nlp(s))
test['wordvec'] = test['nlp'].apply(lambda s: s.vector)

In [8]:
train['keyword_nlp'] = train['cleaned_text_keyword'].apply(lambda s:nlp(s))
train['keyword_wordvec'] = train['keyword_nlp'].apply(lambda s: s.vector)
test['keyword_nlp'] = test['cleaned_text_keyword'].apply(lambda s:nlp(s))
test['keyword_wordvec'] = test['keyword_nlp'].apply(lambda s: s.vector)

## Inspect the generate wordvecs

In [9]:
def check(df, i):
    print(train['text'].iloc[i])
    print(train['cleaned_text'].iloc[i])
    for token in train['nlp'].iloc[i]:
        print(token, token.has_vector)

In [10]:
check(train, 1)

Forest fire near La Ronge Sask. Canada
Forest fire near La Ronge Sask Canada
Forest True
fire True
near True
La True
Ronge False
Sask True
Canada True


In [11]:
check(train, 99)

.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad
Bahrain police had previously died in a road accident they were not killed by explosion
Bahrain True
police True
had True
previously True
died True
in True
a True
road True
accident True
they True
were True
not True
killed True
by True
explosion True


In [12]:
check(train, 888)

@TradCatKnight (1) Russia may have played into reason but that link is BS.  Okanowa was bloody and mainline invasion looked like a bloody
1 Russia may have played into reason but that link is BS Okanowa was bloody and mainline invasion looked like a bloody
1 True
Russia True
may True
have True
played True
into True
reason True
but True
that True
link True
is True
BS True
Okanowa False
was True
bloody True
and True
mainline True
invasion True
looked True
like True
a True
bloody True


### OOV words

In [13]:
oov_tokens = dict()

def gather_tokens(oov_tokens, doc):
    for token in doc:
        if token.is_oov:
            if str(token).lower() in oov_tokens:
                oov_tokens[str(token).lower()] += 1
            else:
                oov_tokens[str(token).lower()] = 1

In [14]:
train['nlp'].apply(lambda x: gather_tokens(oov_tokens,x))
test['nlp'].apply(lambda x: gather_tokens(oov_tokens,x))

0       None
1       None
2       None
3       None
4       None
        ... 
3258    None
3259    None
3260    None
3261    None
3262    None
Name: nlp, Length: 3263, dtype: object

In [15]:
oov_df = pd.DataFrame({'token':list(oov_tokens.keys()), 'number':list(oov_tokens.values())})

In [16]:
oov_df = oov_df.sort_values(by='number',ascending=False)
print(oov_df.head(500).values)

[['mh370' 94]
 ['prebreak' 41]
 ['typhoondevastated' 32]
 ['soudelor' 28]
 ['funtenna' 26]
 ['disea' 25]
 ['gbbo' 23]
 ['udhampur' 21]
 ['bayelsa' 21]
 ['marians' 20]
 ['enugu' 19]
 ['utc20150805' 17]
 ['sensorsenso' 17]
 ['gtgt' 16]
 ['selfimage' 16]
 ['spos' 15]
 ['time20150806' 14]
 ['mtvhottest' 13]
 ['abstorm' 13]
 ['sismo' 13]
 ['bestnaijamade' 12]
 ['mediterran' 12]
 ['hwo' 11]
 ['irandeal' 11]
 ['linkury' 11]
 ['trfc' 11]
 ['okwx' 10]
 ['beyhive' 10]
 ['o784' 10]
 ['meatloving' 10]
 ['yazidis' 10]
 ['wheavenly' 10]
 ['sinjar' 10]
 ['yearold' 10]
 ['summerfate' 10]
 ['inj' 9]
 ['tubestrike' 9]
 ['chicagoarea' 9]
 ['breakingnews' 9]
 ['runion' 9]
 ['animalrescue' 8]
 ['trapmusic' 8]
 ['icemoon' 8]
 ['igers' 8]
 ['olap' 8]
 ['mansehra' 7]
 ['twia' 7]
 ['waterresistant' 7]
 ['explosionproof' 7]
 ['30pm' 7]
 ['pantherattack' 7]
 ['bb17' 7]
 ['zouma' 7]
 ['wisenews' 7]
 ['kisii' 6]
 ['strategicpatience' 6]
 ['auspol' 6]
 ['yycstorm' 6]
 ['abbswinston' 6]
 ['gtgtgt' 6]
 ['kerricktrial

In [17]:
print("Fraction of tokens oov: ", 
      oov_df.number.sum() / (train['nlp'].apply(len).sum() + test['nlp'].apply(len).sum()))

Fraction of tokens oov:  0.04076309050291505


## To explore: use wordninja to cut up composite words/hashtags
Several OOV words seem to be composites that could be cut up. Wordninja can do this. For now just leave as is, since it's only 4% of the words.

In [18]:
import wordninja

In [19]:
wordninja.split('typhoondevastated')

['typhoon', 'devastated']

In [20]:
wordninja.split('mh370')

['mh', '370']

In [21]:
wordninja.split('prebreak')

['pre', 'break']

# Saving the data

The following wordvec data is produced:
* wordvec: wordvec from the tweet (average of the wordvec of all words in the tweet)
* keyword_wordvec: wordvec from keyword_nlp

In [22]:
columns_to_save = ['id', 'wordvec', 'keyword_wordvec']

In [25]:
train[columns_to_save].to_pickle('train_wordvec.pickle')

In [26]:
test[columns_to_save].to_pickle('test_wordvec.pickle')